<div style= "color:CadetBlue; font-family: 'Monaco', 'Lucida Console', 'Courier New', monospace; font-size:16px;padding:10px;">
    
**Problem Statement:**
<span style="color:#FEEA00; text-transform: uppercase;">Predict</span> how many <span style="color:#FEEA00; text-transform: uppercase;">calories</span> were burned during a workout.</p>
**Scoring Criteria:**
Submissions are evaluated using Root Mean Squared Logarithmic Error. (RMSLE).
</div>

# 💾 Initialize and Load Data


In [ ]:
# Import libraries
import warnings
warnings.filterwarnings("ignore")

# Update libraries
!pip install --upgrade scikit-learn
!pip install --upgrade plotly  ## 5.24.1 -> 6.3.1
!pip install --upgrade seaborn  ##  0.12.2 ->  0.12.3

# data manipulation
import numpy as np
import pandas as pd

# import common libraries and toolkits
from multiprocessing import Pool, cpu_count
import urllib.request
#import sys
#import os

# machine learning libraries
import sklearn as skl
import lightgbm as lgb
import xgboost as xgb
import catboost as catb

# visualization libraries
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as go

# time management
import optuna
from time import time
from tqdm import tqdm

# other useful libraries
import math 
from scipy import stats
import itertools
import random

#import pkg_resources
#print("hdbscan:", pkg_resources.get_distribution("hdbscan").version)
#print("sklearn:", skl.__version__)
#print("plotly:", go.__version__)


# reuse my kaggle tabular data functions
url = "https://raw.githubusercontent.com/2awesome-rob/iron_fungi/main/my_kaggle_functions.py"
urllib.request.urlretrieve(url, "my_kaggle_functions.py")
import my_kaggle_functions as mkf

In [ ]:
# specify path
PATH = "/kaggle/input/playground-series-s5e5/"
print(f"Home path is: {PATH}")

# load data
DEVICE, CORES  = mkf.set_globals(verbose = True)
XY, features, targets, target = mkf.load_tabular_data(PATH)

---
# 🧭 Exploratory Data Analysis
## 🔎 Target

In [ ]:
# print target summary stats
mkf.summarize_data(XY, target)

# plot target distribution
mkf.plot_target_eda(XY, target, title = f'{target} distribution')

#### 👀 target observations and notes

Target: Calories (float). Predicted calories burned during workout. </p>

Target distribution has a strong left skew with outliers at higher values

In [ ]:
# add labels for plotting
XY, targets = mkf.get_target_labels(XY, target, targets, cuts=8)

---
## 🔍 Features


In [ ]:
# show feature stats
mkf.summarize_data(XY, features)

In [ ]:
mkf.plot_features_eda(XY, features, target, 'label', 
                      high_label="high", low_label="low")

#### 👀 Feature Observations and Notes

SEVEN predictive features.
 
- 3 numeric / magnitude
    - temp (deg C): distribution skews high with exponential predictability 
    - height (cm): normal distribution, noise obscures signal  
    - weight (kg): normal distribution, noise obscures signal  
- 2 numeric / timedelta
    - age (years): distribution skews young, noise obscures signal
    - duration (min): distribution is uniform, quadradic predictability

- 1 numeric / frequency 
    - heartrate (bpm): normally distributed, with near linear predictability
- 1 object / bool string
    - sex: distribution is balanced, with males over-represented in high calorie trials


---
## 🔰 Out of the Box Performance

In [ ]:
training_features = [f for f in features if 
                     XY[f].dtype!='object']

X_train, y_train, X_val,  y_val, X_test, y_test = mkf.split_training_data(
    XY, training_features, target, validation_size = 0.2
)

In [ ]:
### plot mutual information
mi_scores = mkf.get_feature_mutual_info(X_train, y_train)

In [ ]:
# check feature_importance
model = lgb.LGBMRegressor(verbose=-1, n_jobs=CORES)

feature_importance = mkf.get_feature_importance(
    X_train, X_val, y_train, y_val, task="regression"
)

In [ ]:
single_feature_model, _ = mkf.train_and_score_model(
    X_train[[mi_scores.index[0]]], X_val[[mi_scores.index[0]]], y_train, y_val, 
    model, task="regression_rmsle"
)

In [ ]:
oob_model, _ = mkf.train_and_score_model(
    X_train, X_val, y_train, y_val, 
    model, task="regression_rmsle"
)

#### 👀 Initial Model Observations and Notes
- Single Feature Model is predictive, with lots of noise (RMSLE = 0.1818)
- Model is actually very good out of the box (RMSLE = 0.0771)
- Most |residuals| < 25, but some outliers to > 100
- Untuned LGBM significantly outperforms GaussianNB

---
# 📏 Target Engineering

In [ ]:
### Plot target feature transform functions
mkf.plot_feature_transforms(XY, target)

#### 👀 Target Transform Observations
- [ ] TODO Compare PowerTransform to QuantileTransform appearance

In [ ]:
#Encode the target and plot distribution
XY, targets, TargetTransformer = mkf.get_target_transformer(
    XY, target, targets, name="pwr",
    TargetTransformer=skl.preprocessing.PowerTransformer())

ttarget=targets[-1]

---
# 📐 Feature Engineering
## 🧹 Clean

In [ ]:
### Explore missing values
mkf.plot_null_data(XY, features)

In [ ]:
### Imputer

In [ ]:
### Remove Duplicates
XY = mkf.check_duplicates(XY, features, ttarget, drop=True)

---
## 🧮 Transform Features

In [ ]:
### bool string -> numeric boolean
XY['sex'].replace({'male': 1, 'female': -1}, inplace=True)

In [ ]:
mkf.print_pca_loadings(XY, features)

In [ ]:
#### Generate Expert features before scaling
def get_domain_expert_features(df):
    ### healthy height weight ratio -> PCA 7
    df['bmi'] = df['weight']/ ((df['height']/100) **2) 
    df['height_weight_ratio'] = df['weight']/df['height']
    ###https://www.calculator.net/ideal-weight-calculator.html
    # Male:   48.0 kg + 2.7 kg per inch over 5 feet
    # Female: 45.5 kg + 2.2 kg per inch over 5 feet
    df['male'] = df['sex'].replace({1:1, -1:0})
    df['hanwi'] = df['weight'] - (45.5 + (df['male'] * (48-45.5)) + (2.2 + (df['sex'] * (2.7-2.2)))*(df['height']-152.4)/2.54)
    # Male:	  56.2 kg + 1.41 kg per inch over 5 feet
    # Female: 53.1 kg + 1.36 kg per inch over 5 feet
    df['miller'] = df['weight'] - (53.1 + (df['male'] * (56.2-53.1)) + (1.36 + (df['sex'] * (1.41-1.36)))*(df['height']-152.4))
    df.drop('male', inplace=True, axis=1)
    feats = ['bmi', 'height_weight_ratio', 'hanwi', 'miller']
    df = mkf.get_transformed_features(df, feats, skl.preprocessing.StandardScaler(), winsorize=[0.001, 0.001])
    ### temp vs heart rate -> PCA 5
    df['vital_ratio'] = df['heart_rate'] / df['body_temp']
    ## delta from reference
    df['body_temp'] +=  -37
    df['heart_rate'] +=  -60
    ### effort 
    df['effort'] = df['heart_rate'] * df['body_temp'] * df['duration']
    ### effort per size -> PCA 1
    df['size_effort'] = df['effort'] / (df['weight'] * df['height'])
    ### total effort -> PCA 2
    df['max_effort'] = df['effort'] * df['weight'] * df['height']
    ### workout effort per unit time -> PCA 6
    df['burn_rate'] = df['body_temp'] * df['heart_rate'] / df['duration'] 
    feats = ['effort', 'size_effort', 'max_effort', 'vital_ratio', 'burn_rate']
    df = mkf.get_transformed_features(df, feats , skl.preprocessing.PowerTransformer(), winsorize=[0.001, 0.001])
    XY['temp_2'] = XY['body_temp'] ** 2
    XY['temp_3'] = XY['body_temp'] ** 3
    XY['temp_4'] = XY['body_temp'] ** 4
    feats = ['temp_2', 'temp_3', 'temp_4']
    df = mkf.get_transformed_features(df, feats , skl.preprocessing.MinMaxScaler())
    return df

XY = get_domain_expert_features(XY)

In [ ]:
### Plot scalar feature transform functions
training_features = [f for f in features if f != "sex"]

for f in training_features:
    mkf.plot_feature_transforms(XY, f)

In [ ]:
training_features = ['height', 'weight', 'heart_rate']
XY = mkf.get_transformed_features(XY, training_features, skl.preprocessing.StandardScaler())

training_features = ['body_temp']
XY = mkf.get_transformed_features(XY, training_features, skl.preprocessing.PowerTransformer())

training_features = ['duration']
XY = mkf.get_transformed_features(XY, training_features, skl.preprocessing.MinMaxScaler())

XY['age'] = np.log1p(XY['age'])

---
## ➕ Add Features

In [ ]:
training_features = [features]
XY = mkf.get_feature_interactions(XY, features, winsorize=[0.001, 0.001])

---
## ➖ Dimension Reduction
#### Embeddings

In [ ]:
XY = mkf.get_embeddings(XY, features, 
    skl.decomposition.PCA(n_components=5), "pca_orig_",
    target=target, verbose=True
)

In [ ]:
# 'All' training features
training_features = [f for f in XY.columns if f not in targets and
                    "pca_" not in f]

# rbf features were low info/low importance
#XY = mkf.get_embeddings(XY, training_features, 
#    skl.kernel_approximation.RBFSampler(n_components=16), "rbf_",
#    verbose=False
#)

XY = mkf.get_embeddings(XY, training_features, 
    skl.decomposition.PCA(n_components=10), "pca_all_",
    target=target, verbose=True
)

#### Clustering

In [ ]:
training_features = [f for f in XY.columns if f not in targets and
                     "pca_" in f]

XY = mkf.get_clusters(XY, training_features,
    skl.cluster.KMeans(init="k-means++", n_clusters=5, random_state=69),  "k_means_pca",
    target=ttarget)

In [ ]:
#DBSCAN memory usage was excessive 
#XY = mkf.get_clusters(XY, training_features,
#    skl.cluster.DBSCAN(eps=2, min_samples=333, metric='euclidean', leaf_size=30, n_jobs=CORES), "dbscan_pca", 
#    target=ttarget)

In [ ]:
training_features = [f for f in XY.columns if f not in targets and
                     "pca_" not in f and
                     "scan_" not in f and
                     "k_means_" not in f 
                    ]

XY = mkf.get_clusters(XY, training_features,
    skl.cluster.KMeans(init="k-means++", n_clusters=8, random_state=69),  "k_means_all",
    target=ttarget)

---
## 🟰 Evaluate Performance

In [ ]:
training_features = [f for f in XY.columns if f not in targets and
                     XY[f].dtype!='object']

X_train, y_train, X_val,  y_val, X_test, y_test = mkf.split_training_data(
    XY, training_features, ttarget, validation_size = 0.2
)

In [ ]:
### updated mutual information
mi_scores = mkf.get_feature_mutual_info(X_train, y_train)

In [ ]:
### updated feature information
feature_importance = mkf.get_feature_importance(
    X_train, X_val, y_train, y_val, task="regression"
)

In [ ]:
updated_model, _ = mkf.train_and_score_model(
    X_train, X_val, y_train, y_val, 
    model, task="regression_rmsle", 
    TargetTransformer = TargetTransformer
)

#### 👀 Updated Model Observations and Notes
- Model shows improvement (RMSLE = 0.0771 -> 0.0632)
- [ ] TODO: Investigate outliers in residuals 

---
## 🆖 Outliers

In [ ]:
XY['model_residual'] = XY[ttarget] - updated_model.predict(XY[training_features])
targets.append('model_residual')

In [ ]:
def get_outliers(df, features, target, out_feature, deviations=4, verbose=False):
    df_train = df[df.target_mask.eq(True)]
    m = df_train[out_feature].mean()
    s = df_train[out_feature].std()
    pop = df_train.shape[0]
    print("=" * 69)
    print(f"{pop} samples with mean: {m:.4f} std: {s:.4f}")
    for i in range(1, deviations+1):
        df_outlier = df_train[(df_train[out_feature] > i*(m+s)) | (df_train[out_feature] < i*(m-s))]
        print(f"  {pop - df_outlier.shape[0]} ({100*(1-df_outlier.shape[0]/pop):.2f}%) samples within {i} standard deviation ")
    print("=" * 69)
    print(f"{df_outlier.shape[0]} outliers identified")
    if verbose:
        print(df_outlier.head().T)
    return df_outlier

outliers = get_outliers(XY, features, target, 'model_residual', deviations = 5)

#remove outliers
XY = XY[~XY.index.isin(outliers.index)]

# 🏃‍♂️ Training and Evaluation
## 👯‍♀️ Model Selection

- base_models -> model classes
- models -> base models with hyperparameters
- training_features -> list of training features for each model class

In [ ]:
important_features = [f for f in feature_importance.index.tolist() if 
                      (feature_importance[f] > 2 or
                       mi_scores[f] > 0.2)]

print(f"Not using {[f for f in feature_importance.index.tolist() if f not in important_features]}")


In [ ]:
base_models = {
#    'lr' : skl.linear_model.LogisticRegression, 
    'lgb' : lgb.LGBMRegressor,
    'lgb2' : lgb.LGBMRegressor,
    'xgb' : xgb.XGBRegressor, 
    'xgb2' : xgb.XGBRegressor, 
    'hgb' : skl.ensemble.HistGradientBoostingRegressor,
}

params = {
    'lr' : {'C': 0.6, 'max_iter': 267, 'solver': 'sag'}, 
    'mlp' : {'hidden_layer_sizes' : (64,64), 'random_state' : 69},
#    'hgb' : {'max_iter': 1000, 'learning_rate': 0.038, 'max_leaf_nodes': 24, 'min_samples_leaf': 24, 'l2_regularization': 0.008, 'early_stopping': True}
}

models, training_features = mkf.get_ready_models(XY, important_features, target, 
    base_models, task='regression', direction='minimize', hyper_params=params,
    n_features=3, n_trials=22, CORES=CORES, DEVICE=DEVICE, verbose=False,
)

## 🏋️‍♂️ Model Training

In [ ]:
trained_models, stacking_model = mkf.cv_train_models(XY, training_features, target,
    models, task='regression_rmsle', TargetTransformer=TargetTransformer, 
    folds=11
    )

---
# 🔮 Predict & Submit

In [ ]:
predictions = mkf.submit_cv_predict(X_test, y_test, training_features, target, 
                      trained_models, task='regression_rmsle',
                      meta_model=stacking_model,
                      TargetTransformer=TargetTransformer,
                      path=PATH, verbose=True)
